In [1]:
db_connection = {
    "user": "postgres",
    "password": "*******",
    "host": "localhost",
    "port": "5432",
    "database": "soundsync"
}

In [2]:
import psycopg2
from datetime import datetime

In [3]:
def image_to_bytea(image_path):
    try:
        # Read the image file into bytes
        with open(image_path, 'rb') as f:
            image_data = f.read()

        return image_data

    except Exception as e:
        print("Error:", e)
        return None


In [4]:
def set_profile_pic(userid, image_data):
    # Replace these values with your actual database connection parameters
    
    try:
        # Establish a connection to the PostgreSQL database
        connection = psycopg2.connect(**db_connection)

        # Create a cursor object to interact with the database
        cursor = connection.cursor()

        # Use SQL parameters to prevent SQL injection
        
        # Execute the update query with the image data and userid
        cursor.execute('UPDATE "user" SET profilepic = %s WHERE id = %s', (image_data, userid))

        # Commit the changes to the database
        connection.commit()

        return True, "Profile picture updated successfully"
    except Exception as e:
        return False, f"Error updating profile picture: {str(e)}"
    finally:
        # Close the cursor and connection
        if cursor:
            cursor.close()
        if connection:
            connection.close()


In [20]:
image_data = image_to_bytea('pfp98.jpg')

In [21]:
set_profile_pic(16,image_data)

(True, 'Profile picture updated successfully')

In [3]:
import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity

def recommend(user_id):
    conn = psycopg2.connect(**db_connection)
    cursor = conn.cursor()

    # Fetch the users that the current user is already following
    query_following = f'SELECT following FROM follower_info WHERE follower = {user_id};'
    cursor.execute(query_following)
    users_already_followed = [user[0] for user in cursor.fetchall()]

    query_cluster_label = f'SELECT cluster_label FROM user_clusters WHERE user_id = {user_id};'
    cursor.execute(query_cluster_label)
    cluster_label = cursor.fetchone()[0]

    # Fetch all users within the same cluster
    query_cluster_users = f'SELECT user_id FROM user_clusters WHERE cluster_label = {cluster_label} AND user_id != {user_id};'
    cursor.execute(query_cluster_users)
    users_in_cluster = [user[0] for user in cursor.fetchall()]

    query_cluster_centers = f'SELECT * FROM cluster_centers;'
    cursor.execute(query_cluster_centers)
    rows = cursor.fetchall()
    columns = [desc[0] for desc in cursor.description]

    # Create a DataFrame using the extracted rows and columns
    cluster_centers = pd.DataFrame(rows, columns=columns)
    cluster_centers_without_target = cluster_centers[cluster_centers['cluster_label'] != cluster_label]

    # Get the features for the target cluster
    target_cluster_features = cluster_centers[cluster_centers['cluster_label'] == cluster_label].iloc[:, 1:]

    # Calculate cosine similarity with the remaining clusters
    similarity = cosine_similarity(cluster_centers_without_target.iloc[:, 1:], target_cluster_features)
    cluster_ordering = similarity.sum(axis=1).argsort()[::-1]
    recommended_users = []

    # Add users from the current cluster, excluding those already followed
    recommended_users.extend([user for user in users_in_cluster if user not in users_already_followed])

    if len(recommended_users) >= 5:
        return recommended_users[:5]

    for cluster_index in cluster_ordering:
        cluster_label = cluster_centers_without_target.iloc[cluster_index]['cluster_label']

        # Fetch users from the current cluster
        query_cluster_users = f'SELECT user_id FROM user_clusters WHERE cluster_label = {cluster_label};'
        cursor.execute(query_cluster_users)
        cluster_users = [user[0] for user in cursor.fetchall()]

        # Add users to the recommended list, excluding those already followed
        recommended_users.extend([user for user in cluster_users if user not in users_already_followed])

        # Break if we have enough users
        if len(recommended_users) >= 5:
            break

    conn.close()
    return recommended_users[:5]

In [4]:
def fetch_top_songs(user_id):
    # From song_data fetch all songs whose userid==user_id 
    # From songid(spotify songid) fetch their respective title, albumarturl and artist name from spotipy api 
    # Return a list of songs with this data.

[14, 17, 5, 11, 12]

In [9]:

from spotipy import Spotify
SPOTIPY_CLIENT_ID = '352a916cc74b412bb072ec99a1298376'
SPOTIPY_CLIENT_SECRET = '24fd5cdfc91c4853bdf6a0a38e5e0901'
from spotipy.oauth2 import SpotifyClientCredentials
def fetch_top_songs(user_id):
    # Connect to the database
    conn = psycopg2.connect(**db_connection)
    cursor = conn.cursor()

    # Query to fetch all songs for the given user_id
    query_song_data = f'SELECT songid FROM sound_data WHERE userid = {user_id};'
    cursor.execute(query_song_data)
    song_ids = [song[0] for song in cursor.fetchall()]

    # Close the database connection
    conn.close()

    # Initialize Spotipy client
    sp = Spotify(client_credentials_manager=SpotifyClientCredentials(SPOTIPY_CLIENT_ID, SPOTIPY_CLIENT_SECRET))

    # Fetch additional information for each song from Spotipy
    top_songs = []
    for song_id in song_ids:
        track_info = sp.track(song_id)
        title = track_info['name']
        album_art_url = track_info['album']['images'][0]['url']  # Assuming the first image for simplicity
        artist_name = track_info['artists'][0]['name']  # Assuming the first artist for simplicity

        # Create a dictionary with song information
        song_info = {
            'title': title,
            'album_art_url': album_art_url,
            'artist_name': artist_name
        }

        top_songs.append(song_info)

    return top_songs


In [10]:
mydata = fetch_top_songs(3)
mydata